In [31]:
import pandas as pd
import numpy as np
import torch
import tokenizers
import sys
import torch.nn as nn
import traceback
import random
import torch.nn.functional as F
import os
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from sentence_transformers import SentenceTransformer
import flwr as fl
from collections import OrderedDict
from typing import List, Tuple


os.environ["TOKENIZERS_PARALLELISM"] = "false"
np.random.seed(42)

In [32]:
class DatasetCFG:
    data_root = 'ml-25m'
    num_negatives = 1


class FedCFG:
    num_clients = 10

### 获取训练数据


In [33]:
class MovieLensDataset(Dataset):
    '''
        ratings_data: userId|user_behavior|rating|movieId|title|genres
    '''

    def __init__(self, ratings_data, mode='train'):
        self.user_ids, self.user_behaviors, self.movie_ids, self.movie_titles, self.movie_genres, self.labels = self.generate_dataset(
            ratings_data, mode)

    def __getitem__(self, index):
        return self.user_ids[index], self.user_behaviors[index], self.movie_ids[index], self.movie_titles[index], self.movie_genres[index], self.labels[index]

    def __len__(self):
        return len(self.user_ids)

    def generate_dataset(self, ratings_data, mode='train'):
        movie_id_title_genres_list = list(
            set(zip(ratings_data['movieId'], ratings_data['title'], ratings_data['genres'])))

        user_item_set = set(
            zip(ratings_data['userId'], ratings_data['movieId']))
        user_ids, user_behaviors, movie_ids, movie_titles, movie_genres, labels = [], [], [], [], [], []
        for rating in ratings_data.itertuples():
            user_id = getattr(rating, 'userId')
            user_behavior = getattr(rating, 'user_behavior')
            movie_id = getattr(rating, 'movieId')
            movie_title = getattr(rating, 'title')
            movie_genre = getattr(rating, 'genres')
            if mode == 'train':
                for _ in range(DatasetCFG.num_negatives):
                    negative_movie = random.choice(movie_id_title_genres_list)
                    while (user_id, negative_movie[0]) in user_item_set:
                        negative_movie = random.choice(
                            movie_id_title_genres_list)
                    user_ids.append(str(user_id))
                    user_behaviors.append(user_behavior)
                    movie_ids.append(str(negative_movie[0]))
                    movie_titles.append(negative_movie[1])
                    movie_genres.append(negative_movie[2])
                    labels.append(0)
            user_ids.append(str(user_id))
            user_behaviors.append(user_behavior)
            movie_ids.append(str(movie_id))
            movie_titles.append(movie_title)
            movie_genres.append(movie_genre)
            labels.append(1)
        return user_ids, user_behaviors, movie_ids, movie_titles, movie_genres, labels


def spilt_train_test(file_name='ratings_data_process_0001.csv'):
    ratings_trains = []
    ratings_tests = []

    ratings_data = pd.read_csv(os.path.join(
        DatasetCFG.data_root, file_name))

    ratings_data_group = []
    for name, group in ratings_data.groupby('userId'):
        ratings_data_group.append(group)
    np.random.shuffle(ratings_data_group)

    spilt_size = int(len(ratings_data_group)/FedCFG.num_clients)
    for i in range(FedCFG.num_clients):
        start = i*spilt_size
        end = (i+1)*spilt_size
        ratings_data_tem = pd.concat(ratings_data_group[start:end])
        ratings_train = ratings_data_tem[ratings_data_tem['rank_latest'] != ratings_data_tem.groupby(
            'userId')['rank_latest'].transform('max')]
        ratings_test = ratings_data_tem[ratings_data_tem['rank_latest'] == ratings_data_tem.groupby(
            'userId')['rank_latest'].transform('max')]
        ratings_trains.append(ratings_train)
        ratings_tests.append(ratings_test)
    return ratings_trains, ratings_tests

In [34]:
class Net(nn.Module):
    def __init__(self, device):
        self.device = device
        super(Net, self).__init__()
        self.embedding = SentenceTransformer(
            'models/all_datasets_v4_MiniLM-L6')
        for param in self.embedding.parameters():
            param.requires_grad = False
        self.user_fc1 = nn.Linear(384*2, 512)
        self.user_fc2 = nn.Linear(512, 128)
        self.movie_fc1 = nn.Linear(384*3, 512)
        self.movie_fc2 = nn.Linear(512, 128)
        self.relu = nn.ReLU()

        self.to(device)

    def forward(self, user_id, user_behavior, movie_id, movie_title, movie_genre):
        # display(user_id,user_behavior,movie_id,movie_title,movie_genre)
        user_id_embedding = torch.tensor(
            self.embedding.encode(user_id)).requires_grad_(True)
        user_behavior_embedding = torch.tensor(
            self.embedding.encode(user_behavior)).requires_grad_(True)
        movie_id_embedding = torch.tensor(self.embedding.encode(
            movie_id), requires_grad=True).requires_grad_(True)
        movie_title_embedding = torch.tensor(self.embedding.encode(
            movie_title), requires_grad=True).requires_grad_(True)
        movie_genre_embedding = torch.tensor(self.embedding.encode(
            movie_genre), requires_grad=True).requires_grad_(True)
        # display(user_id_embedding.shape,user_behavior_embedding.shape,movie_id_embedding.shape,movie_title_embedding.shape,movie_genre_embedding.shape)

        user_embedding = torch.cat(
            [user_id_embedding, user_behavior_embedding], dim=1)
        movie_embedding = torch.cat(
            [movie_id_embedding, movie_title_embedding, movie_genre_embedding], dim=1)

        user_embedding = user_embedding.to(self.device)
        movie_embedding = movie_embedding.to(self.device)

        user_out = self.user_fc1(user_embedding)
        user_out = self.relu(user_out)
        user_out = self.user_fc2(user_out)

        movie_out = self.movie_fc1(movie_embedding)
        movie_out = self.relu(movie_out)
        movie_out = self.movie_fc2(movie_out)

        result = user_out*movie_out
        result = F.softmax(torch.sum(result, dim=1), dim=0)

        return result

# Train Net Func


In [35]:
def train(net, train_dataloader, epochs, verbose=True):
    net.train()
    optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
    criterion = nn.BCEWithLogitsLoss()
    for epoch in range(epochs):
        for user_id, user_behavior, movie_id, movie_title, movie_genre, labels in tqdm(train_dataloader):
            optimizer.zero_grad()
            outputs = net(user_id, user_behavior, movie_id,
                          movie_title, movie_genre)
            loss = criterion(outputs, labels.float())
            loss.backward()
            optimizer.step()
        if verbose:
            print('epoch:', epoch, 'loss:', loss.item())


def test(net, test_dataloader):
    net.eval()
    criterion = nn.BCEWithLogitsLoss()
    test_loss = 0
    with torch.no_grad():
        for user_id, user_behavior, movie_id, movie_title, movie_genre, labels in tqdm(test_dataloader):
            outputs = net(user_id, user_behavior, movie_id,
                          movie_title, movie_genre)
            outputs = outputs.to('cpu')
            test_loss += criterion(outputs, labels.float()).item()
    test_loss /= len(test_dataloader)
    print('test_loss:', test_loss)
    return test_loss

# Federated Func


In [36]:
def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]


def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)


class FlowerClient(fl.client.NumPyClient):
    def __init__(self, net, train_dataloader, val_dataloader):
        self.net = net
        self.train_dataloader = train_dataloader
        self.val_dataloader = val_dataloader

    def get_parameters(self, config):
        return get_parameters(self.net)

    def fit(self, parameters, config):
        set_parameters(self.net, parameters)
        train(self.net, self.train_dataloader, epochs=1)
        return get_parameters(self.net), len(self.train_dataloader), {}

    def evaluate(self, parameters, config):
        set_parameters(self.net, parameters)
        loss = test(self.net, self.val_dataloader)
        return float(loss), len(self.val_dataloader), {'loss': loss}


device = torch.device('cpu' if torch.cuda.is_available() else 'cpu')
train_datas, val_datas = spilt_train_test()
train_dataloaders, val_dataloaders = [], []
for i in tqdm(range(FedCFG.num_clients), bar_format="{l_bar}{bar}{r_bar}"):
    train_datasets = MovieLensDataset(train_datas[i], mode='train')
    val_datasets = MovieLensDataset(val_datas[i], mode='test')
    train_dataloaders.append(
        DataLoader(train_datasets, batch_size=64, shuffle=True))
    val_dataloaders.append(
        DataLoader(val_datasets, batch_size=64, shuffle=False))


def client_fn(cid: str) -> FlowerClient:
    # Load model
    net = Net(device)

    train_dataloader = train_dataloaders[int(cid)]
    val_dataloader = val_dataloaders[int(cid)]

    return FlowerClient(net, train_dataloader, val_dataloader)


strategy = fl.server.strategy.FedAvg(
    fraction_fit=1.0,
    fraction_evaluate=0.5,
    min_fit_clients=10,
    min_evaluate_clients=5,
    min_available_clients=10,
)

client_resources = None
if device.type == "cuda":
    client_resources = {"num_gpus": 1, 'num_cpus': 2}

fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=FedCFG.num_clients,
    config=fl.server.ServerConfig(num_rounds=5),
    strategy=strategy,
    client_resources=client_resources,
)

100%|██████████| 10/10 [00:00<00:00, 72.18it/s]
INFO flwr 2023-09-26 08:40:16,490 | app.py:175 | Starting Flower simulation, config: ServerConfig(num_rounds=5, round_timeout=None)
2023-09-26 08:40:20,580	INFO worker.py:1621 -- Started a local Ray instance.
INFO flwr 2023-09-26 08:40:21,990 | app.py:210 | Flower VCE: Ray initialized with resources: {'memory': 122457173812.0, 'object_store_memory': 56767360204.0, 'node:__internal_head__': 1.0, 'node:10.249.41.139': 1.0, 'CPU': 32.0, 'GPU': 2.0, 'accelerator_type:G': 1.0}
INFO flwr 2023-09-26 08:40:21,991 | app.py:218 | No `client_resources` specified. Using minimal resources for clients.
INFO flwr 2023-09-26 08:40:21,991 | app.py:224 | Flower VCE: Resources for each Virtual Client: {'num_cpus': 1, 'num_gpus': 0.0}
INFO flwr 2023-09-26 08:40:22,009 | app.py:270 | Flower VCE: Creating VirtualClientEngineActorPool with 32 actors
INFO flwr 2023-09-26 08:40:22,010 | server.py:89 | Initializing global parameters
INFO flwr 2023-09-26 08:40:22,0

(DefaultActor pid=1231229) epoch: 0 loss: 0.6739499568939209


 98%|█████████▊| 46/47 [02:19<00:03,  3.02s/it] [repeated 3x across cluster]


(DefaultActor pid=1231226) epoch: 0 loss: 0.706369161605835


 60%|█████▉    | 46/77 [02:22<01:35,  3.10s/it] [repeated 12x across cluster]


...

(DefaultActor pid=1231220) epoch: 0 loss: 0.68409663438797


DEBUG flwr 2023-09-26 09:10:00,582 | server.py:173 | evaluate_round 5: strategy sampled 5 clients (out of 10)
100%|██████████| 1/1 [00:00<00:00,  1.22it/s]


(DefaultActor pid=1231222) test_loss: 0.6693738102912903


DEBUG flwr 2023-09-26 09:10:02,941 | server.py:187 | evaluate_round 5 received 5 results and 0 failures
INFO flwr 2023-09-26 09:10:02,942 | server.py:153 | FL finished in 1776.8997298448812
INFO flwr 2023-09-26 09:10:03,016 | app.py:225 | app_fit: losses_distributed [(1, 0.6628390192985535), (2, 0.6679208755493165), (3, 0.6676327347755432), (4, 0.6666861653327942), (5, 0.667325758934021)]
INFO flwr 2023-09-26 09:10:03,019 | app.py:226 | app_fit: metrics_distributed_fit {}
INFO flwr 2023-09-26 09:10:03,020 | app.py:227 | app_fit: metrics_distributed {}
INFO flwr 2023-09-26 09:10:03,022 | app.py:228 | app_fit: losses_centralized []
INFO flwr 2023-09-26 09:10:03,023 | app.py:229 | app_fit: metrics_centralized {}


History (loss, distributed):
	round 1: 0.6628390192985535
	round 2: 0.6679208755493165
	round 3: 0.6676327347755432
	round 4: 0.6666861653327942
	round 5: 0.667325758934021

In [37]:
# trains, tests = spilt_train_test('ratings_data_process_0001.csv')
# display(trains[1].head())

# device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

# ratings_train, ratings_test = spilt_train_test('ratings_data_process_0001.csv')
# train_dataset = MovieLensDataset(ratings_train, mode='train')
# test_dataset = MovieLensDataset(ratings_test, mode='test')

# train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
# test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False)

# model = Net(device)
# optimizer = torch.optim.SGD(
#     filter(lambda p: p.requires_grad, model.parameters()), lr=0.01)
# criterion = nn.MSELoss()

# for epoch in range(5):
#     for batch in tqdm(train_dataloader, total=len(train_dataloader)):
#         user_id, user_behavior, movie_id, movie_title, movie_genre, label = batch
#         label = torch.tensor(label).float().to(device)
#         optimizer.zero_grad()
#         out = model(user_id, user_behavior, movie_id, movie_title, movie_genre)
#         loss = criterion(out, label)
#         loss.backward()
#         optimizer.step()
#     print("epoch:{},loss:{}".format(epoch, loss.item()))

In [38]:
# device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
# model = SentenceTransformer('models/all_datasets_v4_MiniLM-L6')
# for _ in tqdm(range(10000)):
#     text1 = 'dsadsad sdasdasdas dsadas'
#     encode1 = torch.tensor(model.encode(text1, device=device))